# Matching Up the Data
In this notebook, we will identify the relationships between all of the data. Many of the relationships depend on string matches, but the text data is inconsistent and has many typos. For example, *'Assemblymember'* could be written as *'Assemblyman'*, *'Assemblywoman'*, *'A semblmember'*, and more. At the same time, with so many repeated words and phrases, many strings appear to match when they should not. A simple string-distance algorithm might find 'Assemblymember David Chiu' to match with 'Assemblymember Dave Chu', which is not correct. Therefore I use an approach that combines fuzzy string matching and regex  with spacy token similarity and entity linking to match the data.

In [126]:
import pandas as pd
import json
import re
import numpy as np
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

In [3]:
bills = pd.read_csv('ca_leg/legislation_data/bill_tbl.csv')

In [4]:
bill_history = pd.read_csv('ca_leg/legislation_data/bill_history_tbl.csv', dtype={'action_status': str, 'primary_location': str, 'secondary_location': str, 'end_status': str})

In [5]:
bill_versions = pd.read_csv('ca_leg/legislation_data/bill_version_tbl.csv')

In [6]:
with open("ca_leg/legislation_data/bill_version_text.json", "r") as f:
    bill_text = json.load(f)
    authors_data = {bill_id: bill_info['Authors']
                   for bill_id, bill_info in bill_text.items()
                   if 'Authors' in bill_info}


In [7]:
list(set([a for b in [v.keys() for v in authors_data.values()] for a in b]))

['COAUTHOR', 'LEAD_AUTHOR', 'null', 'PRINCIPAL_COAUTHOR']

In [8]:
records = []
for bill_id, authors in authors_data.items():
    for author_type, house in authors.items():
        for house_name, author_name in house.items():
            records.append([bill_id, author_type, "COMMITTEE" if house_name == 'UNKNOWN' else house_name, author_name])

df = pd.DataFrame(records, columns=['bill_id', 'author_type', 'house', 'author_name'])
df['bill_id'] = df['bill_id'].apply(lambda x: re.sub(r'__', '', x))
combined = df.merge(bill_versions, left_on='bill_id', right_on='bill_version_id', how='left')

In [9]:
full = combined.loc[combined['bill_version_action'].notna()].merge(bills, left_on='bill_id_y', right_on='bill_id', how='left')

In [9]:
full.to_csv('ca_leg/legislation_data/combined_table.csv', index=False)

In [10]:
disclosure = pd.read_csv('calaccess/CVR_LOBBY_DISCLOSURE_CD.csv', dtype=str)

In [11]:
expenditure = pd.read_csv('calaccess/LEXP_CD.csv', dtype=str)

In [ ]:
lobbying = disclosure[['FILING_ID', 'FIRM_NAME']].merge(expenditure, on='FILING_ID', how='inner')

0                            NaN
1          1/12/2000 12:00:00 AM
2                            NaN
3                            NaN
4                            NaN
                   ...          
490919    11/13/2024 12:00:00 AM
490920    11/13/0202 12:00:00 AM
490921    11/13/2024 12:00:00 AM
490922    11/13/2024 12:00:00 AM
490923     10/2/2024 12:00:00 AM
Name: EXPN_DATE, Length: 490924, dtype: object

In [147]:
lobbying['EXPN_DATE'] = pd.to_datetime(lobbying['EXPN_DATE'], format='%m/%d/%Y %H:%M:%S %p', errors='coerce')

In [162]:
lb = lobbying.loc[(lobbying['EXPN_DATE'].notna()) & (lobbying['EXPN_DATE'] > pd.to_datetime('2001-01-01', format='%Y-%m-%d')) & ((lobbying['BENE_NAME'].notna()) | (lobbying['BENE_POSIT'].notna()))]
lb.to_csv('calaccess/lobbying_clean.csv', index=False)

In [13]:
assembly_committees = pd.read_csv('pdf_parsing/assembly_committees_clean.csv')

In [14]:
doubles = assembly_committees.loc[assembly_committees['politician'].str.contains(',')]
hyphens = assembly_committees.loc[assembly_committees['politician'].str.contains('-')]
neither = assembly_committees.loc[(~assembly_committees['politician'].str.contains(',')) & (~assembly_committees['politician'].str.contains('-'))]

In [15]:
assembly_roster = pd.read_csv('pdf_parsing/assembly_roster.csv')

In [16]:
doubles[['Last', 'First']] = doubles['politician'].str.split(',', expand=True)
doubles['Last'] = doubles['Last'].str.strip()
doubles['First'] = doubles['First'].str.strip()
doubles.rename(columns={'term': 'Term'}, inplace=True)
dbs = doubles.merge(assembly_roster, on=['Last', 'First', 'Term'], how='left')

In [17]:
hyphens['Last'] = hyphens['politician'].apply(lambda x: re.sub(r'-', ' ', x))
hyp = hyphens.merge(assembly_roster, left_on=['Last', 'term'], right_on=['Last', 'Term'], how='inner')

In [18]:
politicians = pd.concat([neither.merge(assembly_roster, left_on=['politician', 'term'], right_on=['Last', 'Term'], how='inner'), hyp, dbs])

In [109]:
import spacy
from rapidfuzz import fuzz, process
from unidecode import unidecode

In [170]:
def clean_text(text):
    text = unidecode(text.lower().strip())
    return re.sub(r'[^\w\s]', '', text)

df_lob = lb.copy()[['BENE_NAME', 'BENE_POSIT']].drop_duplicates()
df_legislators = politicians[['First', 'Last']].drop_duplicates().apply(lambda x: ' '.join(x), axis=1)
df_legislators = df_legislators.apply(clean_text).drop_duplicates().tolist()
df_committees = politicians['committee_clean'].drop_duplicates().apply(clean_text).tolist()

In [171]:
entity_ids = {name: f"LEG_{i}" for i, name in enumerate(df_legislators)}
entity_ids.update({name: f"COM_{i}" for i, name in enumerate(df_committees)})

In [172]:
nlp = spacy.load("en_core_web_md")

In [173]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = unidecode(text).upper()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    prefixes = r'^(HON|HONORABLE|REP|REPRESENTATIVE|SEN|SENATOR|ASSEMBLY|ASSEMBLYMAN|ASSEMBLYMEMBER|COMMITTEE\s+ON|THE|STAFF\s+OF|OFFICE\s+OF)\s+'
    text = re.sub(prefixes, '', text, flags=re.IGNORECASE)
    suffixes = r'(\s+JR|\s+SR|\s+III|\s+II|\s+IV|\s+MD|\s+PHD|\s+ESQ)$'
    text = re.sub(suffixes, '', text, flags=re.IGNORECASE)
    return text

def preprocess_entity_ids(entity_ids):
    name_mapping = {}
    processed_entities = {}
    ngram_index = {}

    for original_name, entity_id in entity_ids.items():
        clean_name = clean_text(original_name)
        entity_type = 'legislator' if entity_id.startswith("LEG_") else 'committee'

        name_mapping[clean_name] = {
            'original': original_name,
            'id': entity_id,
            'type': entity_type
        }

        processed_entities[original_name] = {
            'clean_name': clean_name,
            'tokens': set(clean_name.split()),
            'entity_type': entity_type,
            'id': entity_id
        }
        tokens = clean_name.split()
        for token in tokens:
            if len(token) >= 3:
                if token not in ngram_index:
                    ngram_index[token] = []
                ngram_index[token].append(original_name)
        if len(tokens) >= 2:
            for i in range(len(tokens) - 1):
                bigram = f"{tokens[i]} {tokens[i+1]}"
                if bigram not in ngram_index:
                    ngram_index[bigram] = []
                ngram_index[bigram].append(original_name)

    return {
        'name_mapping': name_mapping,
        'processed_entities': processed_entities,
        'ngram_index': ngram_index,
        'legislator_names': [name for name, eid in entity_ids.items() if eid.startswith("LEG_")],
        'committee_names': [name for name, eid in entity_ids.items() if eid.startswith("COM_")]
    }

def get_candidates_by_ngrams(text, ngram_index):
    cleaned = clean_text(text)
    tokens = cleaned.split()

    candidates = {}

    for token in tokens:
        if len(token) >= 3 and token in ngram_index:
            for candidate in ngram_index[token]:
                candidates[candidate] = candidates.get(candidate, 0) + 1

    if len(tokens) >= 2:
        for i in range(len(tokens) - 1):
            bigram = f"{tokens[i]} {tokens[i+1]}"
            if bigram in ngram_index:
                for candidate in ngram_index[bigram]:
                    candidates[candidate] = candidates.get(candidate, 0) + 3

    return sorted(candidates.items(), key=lambda x: x[1], reverse=True)

In [177]:
def calculate_similarity_score(text1, text2):
    clean1 = clean_text(text1)
    clean2 = clean_text(text2)

    if not clean1 or not clean2:
        return 0

    if clean1 == clean2:
        return 100

    tokens1 = set(clean1.split())
    tokens2 = set(clean2.split())

    intersection = tokens1.intersection(tokens2)
    jaccard = len(intersection) / (len(tokens1) + len(tokens2) - len(intersection)) if (len(tokens1) + len(tokens2) - len(intersection)) > 0 else 0
    subsequence_score = 0
    if clean1 in clean2 or clean2 in clean1:
        subsequence_score = 30

    fuzzy_score = fuzz.token_set_ratio(clean1, clean2) * 0.7  # FUZZY SCORE

    final_score = (jaccard * 25) + subsequence_score + fuzzy_score # final score

    return min(final_score, 100)

def extract_referenced_names(position_text):
    if not position_text or not isinstance(position_text, str):
        return []
    referenced_names = []
    position_lower = position_text.lower()
    # bulk regex search
    patterns = [
        r'(?:staff|aide|assist\w*|chief|counsel|direct\w*)(?:\s+\w+)?\s+(?:to|for|of|with)\s+(?:sen\w*|rep\w*|assembl\w*|congress\w*)?\s+([A-Za-z\s\.\-]+?)(?:$|,|\s+\(|\s+[A-Z]{2})',
        r'(?:sen\w*|rep\w*|assembl\w*|congress\w*)\s+([A-Za-z\s\.\-]+?)(?:\'s?)?\s+(?:staff|office|aide|assist\w*|chief)',
        r'(?:office|staff)\s+(?:of|for)\s+(?:sen\w*|rep\w*|assembl\w*|congress\w*)?\s+([A-Za-z\s\.\-]+?)(?:$|,|\s+\(|\s+[A-Z]{2})',
        r'(?:sen\w*|rep\w*|assembl\w*|congress\w*)\s+([A-Za-z\s\.\-]{2,30})(?:$|,|\s+\(|\s+[A-Z]{2})',
        r'\b([A-Za-z\s\.\-]{2,30})\s+\([A-Z]{2}\)',
        r'\b(?:senator|representative|congressman|chairperson|chairman|assembl\w*)\s+([A-Za-z\s\.\-]{2,30})\b'
    ]
    for pattern in patterns:
        matches = re.finditer(pattern, position_lower, re.IGNORECASE)
        for match in matches:
            name = match.group(1).strip()
            if name and len(name) > 2:
                if any(term not in name.lower() for term in ['staff', 'office', 'committee']):
                    start, end = match.span(1)
                    original_case = position_text[start:end].strip()
                    if original_case and len(original_case) > 2 and original_case not in referenced_names:
                        referenced_names.append(original_case)

    if not referenced_names: # NER if no matches found
        try:
            doc = nlp(position_text)
            for ent in doc.ents:
                if ent.label_ == "PERSON" and len(ent.text) > 2:
                    if ent.text not in referenced_names:
                        referenced_names.append(ent.text)
        except Exception:
            pass
    return referenced_names

In [ ]:
result_df = df_lob.copy()
entity_data = preprocess_entity_ids(entity_ids)
name_mapping = entity_data['name_mapping']
ngram_index = entity_data['ngram_index']
legislator_names = entity_data['legislator_names']
committee_names = entity_data['committee_names']

GENERIC_LEGISLATOR_TITLES = ['assemblymember', 'senator', 'assemblyman', 'assemblywoman']
result_df['MATCHED_NAME'] = None
result_df['ENTITY_ID'] = None
result_df['ENTITY_TYPE'] = None
result_df['MATCH_METHOD'] = None
result_df['CONFIDENCE'] = None

for idx, row in tqdm(result_df.iterrows(), total=len(result_df)):
    bene_name = str(row['BENE_NAME']) if pd.notna(row['BENE_NAME']) else ""
    bene_position = str(row['BENE_POSIT']) if pd.notna(row['BENE_POSIT']) else ""
    if not bene_name.strip() and not bene_position.strip():
        continue

    position_is_generic = bene_position.strip().lower() in GENERIC_LEGISLATOR_TITLES
    for field_name, field_value in [('BENE_NAME', bene_name),
                                   ('BENE_POSIT', bene_position if not position_is_generic else "")]:
        if not field_value.strip():
            continue

        clean_value = clean_text(field_value)
        if clean_value in name_mapping:
            entity_info = name_mapping[clean_value]
            result_df.at[idx, 'MATCHED_NAME'] = entity_info['original']
            result_df.at[idx, 'ENTITY_ID'] = entity_info['id']
            result_df.at[idx, 'ENTITY_TYPE'] = entity_info['type']
            result_df.at[idx, 'MATCH_METHOD'] = f'exact_{field_name.lower()}'
            result_df.at[idx, 'CONFIDENCE'] = 'high'
            break
    if pd.notna(result_df.at[idx, 'MATCHED_NAME']):
        continue
    all_referenced_names = []
    if not position_is_generic:
        all_referenced_names.extend(extract_referenced_names(bene_position))
    all_referenced_names.extend(extract_referenced_names(bene_name))

    for ref_name in all_referenced_names:
        clean_ref = clean_text(ref_name)
        if clean_ref in name_mapping:
            entity_info = name_mapping[clean_ref]
            result_df.at[idx, 'MATCHED_NAME'] = entity_info['original']
            result_df.at[idx, 'ENTITY_ID'] = entity_info['id']
            result_df.at[idx, 'ENTITY_TYPE'] = entity_info['type']
            result_df.at[idx, 'MATCH_METHOD'] = 'reference_exact'
            result_df.at[idx, 'CONFIDENCE'] = 'high'
            break

    if pd.notna(result_df.at[idx, 'MATCHED_NAME']):
        continue

    combined_text = f"{bene_name} {bene_position}".lower()
    position_has_legislator = any(term in combined_text for term in ['senator', 'representative', 'rep ', 'sen ', 'assemblymember', 'assemblyman', 'assemblywoman'])
    position_has_committee = any(term in combined_text for term in ['committee', 'commission', 'board', 'task force', 'caucus'])
    search_pool = None

    if position_has_legislator:
        search_pool = legislator_names
    elif position_has_committee:
        search_pool = committee_names

    candidates = []
    if bene_name.strip():
        candidates.extend(get_candidates_by_ngrams(bene_name, ngram_index))

    if bene_position.strip() and not position_is_generic:
        candidates.extend(get_candidates_by_ngrams(bene_position, ngram_index))

    seen = set()
    unique_candidates = [(name, score) for name, score in candidates
                         if not (name in seen or seen.add(name))]

    if search_pool:
        unique_candidates = [(name, score) for name, score in unique_candidates if name in search_pool]

    top_candidates = unique_candidates[:10] if unique_candidates else []

    if top_candidates:
        best_match = None
        best_score = 0
        best_method = None

        for candidate_name, _ in top_candidates:
            if bene_name.strip():
                name_score = calculate_similarity_score(bene_name, candidate_name)

                if name_score > best_score and name_score >= 75:
                    best_score = name_score
                    best_match = candidate_name
                    best_method = "fuzzy_name"
            if bene_position.strip() and not position_is_generic:
                position_score = calculate_similarity_score(bene_position, candidate_name)
                if position_score > best_score and position_score >= 80:
                    best_score = position_score
                    best_match = candidate_name
                    best_method = "fuzzy_position"
            if bene_name.strip() and bene_position.strip():
                combined_text = f"{bene_name} {bene_position}"
                combined_score = calculate_similarity_score(combined_text, candidate_name)

                if combined_score > best_score and combined_score >= 70:
                    best_score = combined_score
                    best_match = candidate_name
                    best_method = "fuzzy_combined"

        if best_match:
            confidence = "high" if best_score >= 90 else "medium" if best_score >= 75 else "low"

            result_df.at[idx, 'MATCHED_NAME'] = best_match
            result_df.at[idx, 'ENTITY_ID'] = entity_ids[best_match]
            result_df.at[idx, 'ENTITY_TYPE'] = 'legislator' if entity_ids[best_match].startswith('LEG_') else 'committee'
            result_df.at[idx, 'MATCH_METHOD'] = best_method
            result_df.at[idx, 'CONFIDENCE'] = confidence
            continue

    for field_name, field_value in [('BENE_NAME', bene_name),
                                   ('BENE_POSIT', bene_position if not position_is_generic else "")]:
        if not field_value.strip():
            continue

        clean_value = clean_text(field_value)
        search_list = search_pool if search_pool else list(entity_ids.keys())

        name_match = process.extractOne(
            clean_value,
            search_list,
            scorer=fuzz.token_sort_ratio,
            score_cutoff=85
        )

        if name_match:
            match, score = name_match
            result_df.at[idx, 'MATCHED_NAME'] = match
            result_df.at[idx, 'ENTITY_ID'] = entity_ids[match]
            result_df.at[idx, 'ENTITY_TYPE'] = 'legislator' if entity_ids[match].startswith('LEG_') else 'committee'
            result_df.at[idx, 'MATCH_METHOD'] = f'direct_fuzzy_{field_name.lower()}'
            result_df.at[idx, 'CONFIDENCE'] = 'high' if score > 90 else 'medium'
            break

 40%|███▉      | 18338/46309 [01:54<03:14, 143.66it/s]

In [176]:
result_df

,BENE_NAME,BENE_POSIT,MATCHED_NAME,ENTITY_ID,ENTITY_TYPE,MATCH_METHOD,CONFIDENCE
4759,SEE ATTACHMENT B,NaN,None,None,None,None,None
5315,Gabriela Sanchez,Legislative Staff for Assemblyman Lou Correa,lou correa,LEG_17,legislator,position_exact,high
5346,GIL CEDILLO,ASSEMBLYMEMBER,gil cedillo,LEG_10,legislator,exact_name,high
5347,MARCO ANTONIO FIREBAUGH,ASSEMBLYMEMBER,marco a firebaugh,LEG_23,legislator,fuzzy_name,medium
5348,TONY RICASA,"PRINICIPAL ASSISTANT, ASSEMBLYMEMBER CEDILLO",None,None,None,None,None
...,...,...,...,...,...,...,...
490917,Gianna Setoudeh,Senior Legislative Aide to Senator Caballero,None,None,None,None,None
490918,TINA McKINNOR,ASSEMBYWOMAN,tina s mckinnor,LEG_343,legislator,fuzzy_name,medium
490919,TINA McKINNOR,ASSEMBLYWOMAN,tina s mckinnor,LEG_343,legislator,fuzzy_name,medium
490922,MAYA DOUGLAS,DISTRICT DIRECTOR ASSEMBLYMAN MIKE GIPSON,None,None,None,None,None
